# Decision Tree Regression 2

We zullen in deze notebook een Decision Tree model maken die de reparatietijd voorspelt op base van de uitgebreide gekozen features *oorzaak groep, oorzaak code, prioriteit.

In [1]:
#import benodigde libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#De opgeschone dataset inlezen
data2 = pd.read_pickle('data2.pkl')
data2.head()

stm_prioriteit  reparatietijd  stm_oorz_groep_ONR-DERD  \
32099             9.0      40.000000                        0   
86473             9.0     228.650000                        0   
91875             9.0     220.316667                        1   
92852             9.0      55.000000                        0   
93767             9.0      15.233333                        1   

       stm_oorz_groep_ONR-RIB  stm_oorz_groep_TECHONV  stm_oorz_groep_WEER  \
32099                       0                       1                    0   
86473                       1                       0                    0   
91875                       0                       0                    0   
92852                       0                       1                    0   
93767                       0                       0                    0   

       stm_oorz_code_130.0  stm_oorz_code_131.0  stm_oorz_code_132.0  \
32099                    0                    0                    0   
86473                    0                    0                    0   
91875                    0                    0                    0   
92852                    0                    0                    0   
93767                    0                    0                    0   

       stm_oorz_code_133.0  ...  stm_sap_meldtijd_uren_14  \
32099                    0  ...                         0   
86473                    1  ...                         0   
91875                    0  ...                         1   
92852                    0  ...                         0   
93767                    0  ...                         0   

       stm_sap_meldtijd_uren_15  stm_sap_meldtijd_uren_16  \
32099                         0                         0   
86473                         0                         1   
91875                         0                         0   
92852                         0                         0   
93767                         0                         0   

       stm_sap_meldtijd_uren_17  stm_sap_meldtijd_uren_18  \
32099                         0                         0   
86473                         0                         0   
91875                         0                         0   
92852                         0                         0   
93767                         0                         0   

       stm_sap_meldtijd_uren_19  stm_sap_meldtijd_uren_20  \
32099                         0                         0   
86473                         0                         0   
91875                         0                         0   
92852                         0                         0   
93767                         0                         0   

       stm_sap_meldtijd_uren_21  stm_sap_meldtijd_uren_22  \
32099                         1                         0   
86473                         0                         0   
91875                         0                         0   
92852                         0                         0   
93767                         0                         0   

       stm_sap_meldtijd_uren_23  
32099                         0  
86473                         0  
91875                         0  
92852                         0  
93767                         0  

[5 rows x 148 columns]

In [3]:
#Herbruikbare functie die een Decision tree Regressor model bouwt 
def decisionTreeRegressor(features, target):
    x_train, x_test, y_train, y_test = train_test_split(features, target) # splitsen in traiining en testing dataset
    # Geeft de dimensie van de opgesplitste datasets
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    treereg = DecisionTreeRegressor()
    #fit model 
    treereg.fit(x_train, y_train)
    # Make prediction
    y_pred = treereg.predict(x_test)
    #Evaluate mbh van RMSE
    trscore = np.sqrt(mse(y_test, y_pred))
    return treereg, trscore

In [4]:
#data opslitsen in features en target 
features = data2.drop('reparatietijd', axis = 1)
target = data2['reparatietijd']

In [5]:
#Decision tree regressor toepassen op features en target 
treereg,trscore = decisionTreeRegressor(features, target)
trscore

(211890, 147)
(211890,)
(70631, 147)
(70631,)


82.52102460549678

# Validatie model 

In [9]:
## Validatie model
# Vaidatieset nlezen en geschikt maken omte voospellen
vadata = pd.read_csv('C:\Amaryllis_school_documents\Jaar_20_21\DDB\ddb_prorail\DDB_Prorail\data\Validatieset 2020-2021 CSV.csv', delimiter= ';')
data2 = vadata[['stm_oorz_groep', 'stm_oorz_code', 'stm_prioriteit', 'stm_contractgeb_gst', 'stm_sap_meldtijd']].copy()
data2 = pd.get_dummies(data2, columns = ['stm_oorz_groep'])
data2 = pd.get_dummies(data2, columns = ['stm_oorz_code'])
data2 = pd.get_dummies(data2, columns = ['stm_contractgeb_gst'])
data2['stm_sap_meldtijd_uren'] = round(pd.to_timedelta(data2['stm_sap_meldtijd']).dt.total_seconds().div(3600)).astype(int)
data2 = data2.drop('stm_sap_meldtijd', axis  = 1)
data2 = pd.get_dummies(data2, columns = ['stm_sap_meldtijd_uren'])
data3 = pd.DataFrame(data=None, columns=features.columns)
data3 = data3.append(data2)
data3 = data3.fillna(0)

#Validatieset voorspellen 
y_pred = treereg.predict(data3)

#Voorspelde waardes printen
print("Voorspellinggen: {}".format(y_pred))

# De reparatietijd van de validatieset berekenen
data4 = vadata[['stm_aanntpl_dd','stm_aanntpl_tijd','stm_fh_ddt']].copy()
data4.loc[:, 'reparatietijd'] = (pd.to_datetime(data4['stm_fh_ddt']) -  pd.to_datetime(data4.loc[:,'stm_aanntpl_dd']  +" "+ data4['stm_aanntpl_tijd'])).dt.total_seconds().div(60)

#RMSE score bepalen 
validatiescore = np.sqrt(mse(data4['reparatietijd'], y_pred))
validatiescore

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Voorspellinggen: [178.01666667 395.         329.38333333]


288.60587969806903

In [10]:
# Actuale waardes zien
data4['reparatietijd']

0    14.983333
1     0.016667
2    69.983333
Name: reparatietijd, dtype: float64